In [11]:
f = open("dataset_865452_14.txt","r")

MATCH_REWARD, MISMATCH_PENALTY, INDEL_PENALTY = [int(x) for x in next(f).split()]
s, t = f.readline().strip('\n'), f.readline().strip('\n')
f.close()

def InterpretAlignment(s, t, alignment):
    i, j = 0, 0
    score = 0
    aligned1 = ""
    aligned2 = ""
    for a in alignment:
        if a == 'D':
            aligned1 += s[i]
            aligned2 += t[j]
            if s[i] == t[j]:
                score += MATCH_REWARD
            else:
                score -= MISMATCH_PENALTY
            i += 1
            j += 1
        elif a == 'V':
            aligned1 += s[i]
            aligned2 += '-'
            score -= INDEL_PENALTY
            i += 1
        else:
            aligned1 += '-'
            aligned2 += t[j]
            score -= INDEL_PENALTY
            j += 1
            
    return score, aligned1, aligned2
                

def LinearSpaceAlignment(s, t, top, bottom, left, right):
    if left == right:
        return str('V' * (bottom-top))
    if top == bottom:
        return str('H' * (right-left))
    
    middle = int((left + right)/2)
    midEdge = MiddleEdge(s[top:bottom], t[left:right], top, bottom, left, right)
    midNode = midEdge[0][0] + top
    
    out = ""
    out += LinearSpaceAlignment(s, t, top, midNode, left, middle)
    
    if midEdge[1][0] - midEdge[0][0] == 1:
        if midEdge[1][1] - midEdge[0][1] == 1:
            out += 'D'
        else:
            out += 'V'
    else:
        out += 'H'
    
    if out[-1] == 'H' or out[-1] == 'D':
        middle += 1
    if out[-1] =='V' or out[-1] == 'D':
        midNode += 1
    
    return out + LinearSpaceAlignment(s, t, midNode, bottom, middle, right)


def MiddleEdge(s, t, top, bottom, left, right):
    #n, m = len(s) + 1, len(t) + 1
    n, m = bottom - top + 1, right - left + 1

    # FromSource(i)
    column1 = [0 - INDEL_PENALTY * i for i in range(n)]
    column2 = [0 for i in range(n)]
    for t_index in range(int(len(t)/2)):
        column2[0] = -(INDEL_PENALTY * (t_index+1))
        for i in range(1,n):
            match = MATCH_REWARD if s[i-1] == t[t_index] else -MISMATCH_PENALTY
            column2[i] = max(column1[i-1] + match, column1[i] - INDEL_PENALTY, column2[i-1] - INDEL_PENALTY)
            
        column1 = column2.copy()
    FromColumn1 = column1
    
    # ToSink(i)
    s_reverse = s[::-1]
    t_reverse = t[::-1]
    column1_reverse = [0 - INDEL_PENALTY * i for i in range(n)]
    column2_reverse = [0 for i in range(n)]
    temp_column = [0 for i in range(n)]
    for t_index in range(int(len(t)/2) + (1 if len(t)%2 == 1 else 0)):
        temp_column = column1_reverse.copy()
        column2_reverse[0] = -(INDEL_PENALTY * (t_index+1))
        for i in range(1,n):
            match = MATCH_REWARD if s_reverse[i-1] == t_reverse[t_index] else -MISMATCH_PENALTY
            column2_reverse[i] = max(column1_reverse[i-1] + match, column1_reverse[i] - INDEL_PENALTY, \
                                     column2_reverse[i-1] - INDEL_PENALTY)
        
        column1_reverse = column2_reverse.copy()
    ToColumn1 = column1_reverse[::-1]
    temp_column = temp_column[::-1]
    
    middle_nodes1 = [FromColumn1[i] + ToColumn1[i] for i in range(n)]
    middle_i1 = -1
    max_node_value1 = max(middle_nodes1)
    for i in range(len(middle_nodes1)):
        if middle_nodes1[i] == max_node_value1:
            middle_i1 = i

    mid = int(len(t)/2)
    if middle_i1 == n - 1:
        return ((middle_i1, mid),(middle_i1,mid+1))
    else:
        match = MATCH_REWARD if s[middle_i1] == t[mid] else -MISMATCH_PENALTY
        if temp_column[middle_i1+1] == ToColumn1[middle_i1] - match:
            return ((middle_i1,mid),(middle_i1+1,mid+1))
            
        if temp_column[middle_i1] == ToColumn1[middle_i1] + INDEL_PENALTY:
            return ((middle_i1,mid),(middle_i1,mid+1))
        else:
            return ((middle_i1,mid),(middle_i1+1,mid))
    
        
        
for item in InterpretAlignment(s, t, LinearSpaceAlignment(s, t, 0, len(s), 0, len(t))):
    print(item)


-136
GCAGCTCGACCTGTCGTAATAGTTAGGCAAGTTGAAAAGATGGTGTTAATACAGTGTGTCGCTACAACCGTGTCACGA-ACGTTACGGTCG-ATGAATGCAATCCTCGTCAGATCTTATGCCCCCCAAGGAGTATCGTAGTAGGCCGAAATGAAAATTATCGACTTAGACGGGGTTTAGCCAGGGCGGTATGCAGAACAGAGCGTCTAGAGACGCTGACGATCGGGTCT----G----ATCCGCTTAGTAGTTCGAATGTAGGTAACTGTGGTCTACCTGCAGGCTTTGCGGACA-CTTG-TCA-TCG-CATAAGGCTGGG-GCCGAGATTAGCGTTGCGTGTAACTTCAAATGATTTGTAGGAAGCATTAGCGAGATACATCCACT-TTCA-AGT-GCG-CTTAGCGCGGT-A-CTCGATAGTTATTAACACTGACTAGAGCCTAACATGA-CCGCCTAGTACCTCGGCGCACTTATCTTTGCAGGCG-GG-ATGCGCCATTATTTCATATATGGTCGAT--A--GTGGCCAACCCCGCGTCA-CTT-ACGTGTACGCAGGGAA-TTATGAGTTCATCCACCCCGAGTGATGTGCACTCCTGCCCCTTT---TTATATCCGTGTTGTTGGGCTACGCAAGGAAGAGCCATAAACAATCCCGAAACAGAGCACAACACAGGGGCTGTCAG-A-CA--C-CCC---GGGTCCGGGGTTTT-A-GCTCCAAAGCAAACCGGCACTAGGAGGTGTATATCATTTGGG-TCA-GAGCCAC-ATGGTAAT--TAATATAAATATGGCTGACGACATTGTGT-TCA-CGCCATCACGAAACGAGACATGATCCTGAATGCCCACGCAAGCAGCCTTTAAC---TGGTGACTCATACGCGTACACCTCTTTT--T-TG-TCAAGGGGCTCGTCTATCTCATTCGGGACACGCTAATCCAGATTGTATTGAGTTCCCGGGGAACGTCCACGTAGTGAGTCC-C